In [0]:
-- Polk probability of purchase a vehicle in market in the next 6 months
DECLARE VARIABLE IN_MARKET_PROB1 INT DEFAULT 30;
DECLARE VARIABLE IN_MARKET_PROB2 INT DEFAULT 30;
DECLARE VARIABLE OWNER_SCORE INT DEFAULT 1;
DECLARE VARIABLE LOYALTY_SCORE INT DEFAULT 1;


CREATE OR REPLACE TABLE infact_dev.marsci.silverado_audience_media AS
(
WITH GM_curr_customer AS (
SELECT gm_person_real_id
FROM (
    SELECT 
        gm_person_real_id, 
        indiv_id, 
        nameplate
    FROM infact_dev.marsci.output_develop_fs_pronghorn_xref
    RIGHT JOIN (
        SELECT 
            acq.individual_business_id, 
            acq.vehicle_status_cd, 
            acq.acquisition_dt, 
            LOWER(info.umf_xref_finc_brand) AS nameplate
        FROM acquire.vehicle_silver.customer_vin_relation AS acq
        LEFT JOIN reporting.vehicle360.fs_veh_ident AS info
            ON acq.vin = info.veh_id_nbr
        WHERE acq.vehicle_status_cd LIKE '%SR%'         -- Bought new filter
          AND acq.individual_business_id LIKE '1%'        -- Retail sale filter
          AND ctry_of_sale = 'US'
    ) AS sub
        ON indiv_id = individual_business_id
)
WHERE nameplate IS NOT NULL
GROUP BY gm_person_real_id
)

, silverado_media AS (
SELECT DISTINCT
GM_PERSON_REALID as gm_person_real_id
FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
WHERE (
PM020157 <= IN_MARKET_PROB1 -- Polk in market prob for Ford F150
OR PM020158 <= IN_MARKET_PROB1 -- Polk in market prob for Ford F250
-- OR PM020171 <= IN_MARKET_PROB1 -- Polk in market prob for GMC Sierra
OR PM020281 <= IN_MARKET_PROB1 -- Polk in market prob for RAM 1500
OR PM020282 <= IN_MARKET_PROB1 -- Polk in market prob for RAM 2500
OR PM020283 <= IN_MARKET_PROB1 -- Polk in market prob for RAM 3500
OR PM020308 <= IN_MARKET_PROB1 -- Polk in market prob for Toyota Tundra
OR PM020274 <= IN_MARKET_PROB1 -- Polk in market prob for Nissan Titan
OR PM020275 <= IN_MARKET_PROB1 -- Polk in market prob for Nissan Titan XD
-- 80% 35M
-- 90% 19M
-- 95% 7.2M
-- 99% 1.9M
)
AND
PM020379 <= LOYALTY_SCORE -- Polk pickup loyalty score
)


SELECT gm_person_real_id FROM silverado_media
WHERE gm_person_real_id 
NOT IN 
(SELECT gm_person_real_id 
FROM GM_curr_customer 
WHERE gm_person_real_id IS NOT NULL)

);


-- top 3 deciles


CREATE OR REPLACE TABLE infact_dev.marsci.silverado_audience_crm AS
(
WITH GM_curr_customer AS (
SELECT gm_person_real_id
FROM (
    SELECT 
        gm_person_real_id, 
        indiv_id, 
        nameplate
    FROM infact_dev.marsci.output_develop_fs_pronghorn_xref
    RIGHT JOIN (
        SELECT 
            acq.individual_business_id, 
            acq.vehicle_status_cd, 
            acq.acquisition_dt, 
            LOWER(info.umf_xref_finc_brand) AS nameplate
        FROM acquire.vehicle_silver.customer_vin_relation AS acq
        LEFT JOIN reporting.vehicle360.fs_veh_ident AS info
            ON acq.vin = info.veh_id_nbr
        WHERE acq.vehicle_status_cd LIKE '%SR%'         -- Bought new filter
          AND acq.individual_business_id LIKE '1%'        -- Retail sale filter
          AND ctry_of_sale = 'US'
    ) AS sub
        ON indiv_id = individual_business_id
)
WHERE nameplate IS NOT NULL
GROUP BY gm_person_real_id
)

, silverado_CRM AS (
SELECT DISTINCT
GM_PERSON_REALID as gm_person_real_id
FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
WHERE (
PM021434 <= OWNER_SCORE -- Polk owner score for Ford F150
OR PM021435 <= OWNER_SCORE -- Polk owner score for Ford F250
-- OR PM020840 <= OWNER_SCORE -- Polk owner score for GMC Sierra
OR PM021030 <= OWNER_SCORE -- Polk owner score for RAM 1500
OR PM021031 <= OWNER_SCORE -- Polk owner score for RAM 2500
OR PM021032 <= OWNER_SCORE -- Polk owner score for RAM 3500
OR PM021078 <= OWNER_SCORE -- Polk owner score for Toyota Tundra
OR PM021010 <= OWNER_SCORE -- Polk owner score for Nissan Titan
)
AND (
PM020157 <= IN_MARKET_PROB2 -- Polk in market prob for Ford F150
OR PM020158 <= IN_MARKET_PROB2 -- Polk in market prob for Ford F250
-- OR PM020171 <= IN_MARKET_PROB2 -- Polk in market prob for GMC Sierra
OR PM020281 <= IN_MARKET_PROB2 -- Polk in market prob for RAM 1500
OR PM020282 <= IN_MARKET_PROB2 -- Polk in market prob for RAM 2500
OR PM020283 <= IN_MARKET_PROB2 -- Polk in market prob for RAM 3500
OR PM020308 <= IN_MARKET_PROB2 -- Polk in market prob for Toyota Tundra
OR PM020274 <= IN_MARKET_PROB2 -- Polk in market prob for Nissan Titan
OR PM020275 <= IN_MARKET_PROB2 -- Polk in market prob for Nissan Titan XD
)
-- AND
-- PM020379 <= LOYALTY_SCORE -- Polk pickup loyalty score
-- 80% 5.1M
-- 90% 3.9M
-- 95% 2.2M
-- 99% 0.8M
)

SELECT gm_person_real_id FROM silverado_CRM
WHERE gm_person_real_id 
IN 
(SELECT gm_person_real_id 
FROM GM_curr_customer 
WHERE gm_person_real_id IS NOT NULL)

)





